# Deep Neural Network for MNIST Classification

## Import the relevant packages

In [1]:
import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds

## Data

In [2]:
mnist_dataset, mnist_info = tfds.load(name='mnist', with_info=True, as_supervised=True)

In [36]:
mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples

num_validation_samples = tf.cast(num_validation_samples, tf.int64)

num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)

def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255.
    return image, label

scaled_train_and_validation_data = mnist_train.map(scale)
test_data = mnist_test.map(scale)

BUFFER_SIZE = 10000
shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

validation_data = shuffled_train_and_validation_data.take(num_validation_samples)
train_data = shuffled_train_and_validation_data.skip(num_validation_samples)

BATCH_SIZE = 100
train_data = train_data.batch(BATCH_SIZE)
validation_data = validation_data.batch(num_validation_samples)
test_data = test_data.batch(num_test_samples)

validation_inputs, validation_targets = next(iter(validation_data))

## Model

### Outline the model

In [57]:
input_size = 784
output_size = 10
hidden_layer_size = 300

model = tf.keras.Sequential([
                            tf.keras.layers.Flatten(input_shape=(28,28,1)),
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                            tf.keras.layers.Dense(hidden_layer_size, activation='sigmoid'),
                            tf.keras.layers.Dense(output_size, activation='softmax')
                            ])

### Choose the optimizer and the loss function

In [58]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

### Training

In [60]:
NUM_EPOCHS = 5

model.fit(train_data, epochs = NUM_EPOCHS, validation_data=(validation_inputs, validation_targets), verbose=2)

Epoch 1/5
540/540 - 1s - loss: 0.0312 - accuracy: 0.9907 - val_loss: 0.0401 - val_accuracy: 0.9883
Epoch 2/5
540/540 - 1s - loss: 0.0240 - accuracy: 0.9925 - val_loss: 0.0319 - val_accuracy: 0.9908
Epoch 3/5
540/540 - 1s - loss: 0.0203 - accuracy: 0.9938 - val_loss: 0.0309 - val_accuracy: 0.9905
Epoch 4/5
540/540 - 1s - loss: 0.0166 - accuracy: 0.9950 - val_loss: 0.0214 - val_accuracy: 0.9940
Epoch 5/5
540/540 - 1s - loss: 0.0127 - accuracy: 0.9963 - val_loss: 0.0144 - val_accuracy: 0.9972


### Test the Model

In [62]:
test_loss, test_accuracy = model.evaluate(test_data)

1/1 [==============================] - 0s 391ms/step - loss: 0.0682 - accuracy: 0.9812


In [63]:
print('Test loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))

Test loss: 0.07. Test accuracy: 98.12%
